# I. Lý thuyết

1) Transfer Learning thường được thực hiện trên hai dữ liệu nguồn (huấn luyện mô hình nguồn) và dữ liệu đích (huấn luyện mô hình đích) thế nào ?


A. Dữ liệu nguồn và dữ liệu đích có sự liên quan tới nhau. Những đặc trưng trong dữ liệu đích xuất hiện ở những dữ liệu nguồn.

B. Dữ liệu nguồn có số lượng classes lớn hơn dữ liệu đích.

C. Kích thước của dữ liệu nguồn rất nhỏ.

D. Dữ liệu đích ít liên quan tới dữ liệu nguồn.

ĐA: A


2) Khi nào thì chúng ta nên thực hiện fine tuning trên toàn bộ các layers của mô hình đích ?

A. Khi hai dữ liệu nguồn và đích có mức độ tương đồng cao và dữ liệu đích có kích thước lớn.

B. Khi hai dữ liệu nguồn và đích có mức độ tương đồng cao và dữ liệu đích có kích thước nhỏ.

C. Dữ liệu đích có kích thước nhỏ và dữ liệu nguồn có kích thước lớn.

D. Cả hai dữ liệu đích và nguồn đều có kích thước nhỏ.


ĐA: A


3) Các phương pháp augmentation nào sẽ kết hợp nội dung từ hai ảnh lẫn nhau và tạo ra một nhãn mềm (_soft label_) cho ảnh?

A. Rotation, Random Crop, MixUp

B. Bright Constrast, Color Shift, Addition Noise

C. CutMix, MixUp

D. Flip, Information Loss 


ĐA: C


4) Quá trình xây dựng một mô hình AI trong dự án là một chu trình Machine Learning Cycle kế hợp giữa huấn luyện và gán nhãn dữ liệu. Để tiết kiệm chi phí gán nhãn chúng ta nên sử dụng phương pháp nào ?

A. Lấy mẫu ngẫu nhiên từ tập unlabeled dataset để thực hiện gán nhãn.

B. Sử dụng Active Learning để lựa chọn mẫu mang lại thông tin giúp cải thiện nhiều nhất cho hiệu suất mô hình.

C. Lựa chọn mô hình pretrained lớn nhất có thể.

D. Chỉ lựa các dữ liệu có thông tin rõ ràng, có thể phân biệt được bởi con người.


ĐA: B

5) Mô hình lớn thường đạt độ chính xác cao nhưng không deploy được trên các thiết bị edge device, IoT,... Trong khi mô hình nhỏ có thể deploy được nhưng thường có độ chính xác thấp. Phương pháp nào có thể giúp mô hình nhỏ cải thiện được độ chính xác ? Có thể lựa chọn nhiều đáp án.

A. Sử dụng active learning để lựa chọn các mẫu đại diện cho tổng thể để huấn luyện mô hình nhỏ.

B. Áp dụng augmentation để huấn luyện mô hình nhỏ.

C. Fine tuning các layers của mô hình lớn sang mô hình nhỏ.

D. Sử dụng knowledge distillation để chuyển giao tri thức từ mô hình lớn sang mô hình nhỏ.

ĐA: B, C, D

# II. Thực hành

# 6) Từ bộ dữ liệu [Dog and Cat](https://www.kaggle.com/c/dog-vs-cat-classification/data), hãy huấn luyện một mô hình large (chẳng hạn ResNet50) bằng cách fine-tuning lại các trọng số từ pretrained model của bộ dữ liệu ImageNet. Huấn luyện trên 5 epochs.


In [ ]:
from google.colab import drive
import os
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/DeepLearning2/6.TransferLearning

/content/gdrive/MyDrive/Colab Notebooks/DeepLearning2/6.TransferLearning


In [3]:
import os
import zipfile
import glob
import time
import pickle
import pprint
import math

%matplotlib inline
import matplotlib.pyplot as plt
import PIL

import numpy as np
import pandas as pd
import sklearn.model_selection

import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torch.utils.data
from torch.utils.data import Dataset, DataLoader

import tqdm
import random
import torchvision.transforms as T

from sklearn.model_selection import train_test_split

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
device

'cuda'

In [ ]:
TRAIN_DIR = '../data/dogvscat/train/train'
TEST_DIR = '../data/dogvscat/test/test'
train_images = glob.glob(TRAIN_DIR+"/**/**.jpg")
test_images = glob.glob(TEST_DIR+"/**.jpg")
print(len(train_images), len(test_images))

25000 8000


In [ ]:
dogs_list = [img for img in train_images if img.split("/")[-2] == "dogs"]
cats_list = [img for img in train_images if img.split("/")[-2] == "cats"]

print("Dogs Images: ",len(dogs_list))
print("Cats Images: ",len(cats_list))

class_to_int = {"dogs" : 0, "cats" : 1}
int_to_class = {0 : "dogs", 1 : "cats"}

Dogs Images:  12500
Cats Images:  12500


In [ ]:
def get_train_transform():
    return T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(15),
        T.Resize((256, 256)),
        T.RandomResizedCrop(224),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])
    
def get_val_transform():
    return T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

In [ ]:
from PIL import Image
class CatDogDataset(Dataset):
    def __init__(self, imgs, class_to_int, mode = "train", 
                 transforms = None):
        super().__init__()
        self.imgs = imgs
        self.class_to_int = class_to_int
        self.mode = mode
        self.transforms = transforms
    def __getitem__(self, idx):
        image_name = self.imgs[idx]
        if self.mode == "train" or self.mode == "val":
            img = Image.open(image_name)
            # img = img.resize((256, 256))
            ### Preparing class label
            label = self.class_to_int[image_name.split("/")[-2]]
            label = torch.tensor(label, dtype = torch.float32)
            ### Apply Transforms on image
            img = self.transforms(img)
            return img, label
        elif self.mode == "test":
            img = Image.open(image_name)
            # img = img.resize((256, 256))
            ### Apply Transforms on image
            img = self.transforms(img)
            return img, image_name
    def __len__(self):
        return len(self.imgs)

In [ ]:
train_imgs, val_imgs = train_test_split(train_images, test_size = 0.2)

In [ ]:
train_dataset = CatDogDataset(train_imgs, class_to_int, mode = "train", transforms = get_train_transform())
val_dataset = CatDogDataset(val_imgs, class_to_int, mode = "val", transforms = get_val_transform())
test_dataset = CatDogDataset(test_images, class_to_int, mode = "test", transforms = get_val_transform())

train_data_loader = DataLoader(
    dataset = train_dataset,
    num_workers = 4,
    batch_size = 32,
    shuffle = True
)

val_data_loader = DataLoader(
    dataset = val_dataset,
    num_workers = 4,
    batch_size = 32,
    shuffle = True
)

test_data_loader = DataLoader(
    dataset = test_dataset,
    num_workers = 4,
    batch_size = 1,
    shuffle = False
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def accuracy(preds, trues):
    preds = [1 if preds[i] >= 0.5 else 0 for i in range(len(preds))]
    acc = [1 if preds[i] == trues[i] else 0 for i in range(len(preds))]
    acc = np.sum(acc) / len(preds)
    return (acc * 100)

In [ ]:
def train_one_epoch(train_data_loader):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()

    model.train()
    
    for images, labels in train_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Reseting Gradients
        optimizer.zero_grad()
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
        
        #Backward
        _loss.backward()
        optimizer.step()
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [ ]:
def val_one_epoch(val_data_loader, best_val_acc, model_name):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()

    model.eval()
    
    for images, labels in val_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    val_logs["loss"].append(epoch_loss)
    val_logs["accuracy"].append(epoch_acc)
    val_logs["time"].append(total_time)
    
    ###Saving best model
    if epoch_acc > best_val_acc:
        best_val_acc = epoch_acc
        torch.save(model.state_dict(),model_name+"_best.pth")
        
    return epoch_loss, epoch_acc, total_time, best_val_acc

In [9]:
from torchvision.models import resnet18
teacher = resnet18(pretrained = True)

# Modifying Head - classifier

teacher.fc = nn.Sequential(
    nn.Linear(512, 1, bias = True),
    nn.Sigmoid()
)

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(teacher.parameters(), lr = 0.0005)

# Learning Rate Scheduler: Learning rate decay double time after each 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# Loading model to device
teacher.to(device)

# No of epochs 
epochs = 5

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, best_val_acc, "resnet50")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Training
Epoch 1
Loss : 0.5719
Acc : 70.215
Time : 157.9923

Validating
Epoch 1
Loss : 0.5523
Acc : 72.0143
Time : 26.4719

Training
Epoch 2
Loss : 0.568
Acc : 70.41
Time : 148.2375

Validating
Epoch 2
Loss : 0.5517
Acc : 71.9347
Time : 26.4737

Training
Epoch 3
Loss : 0.5729
Acc : 70.445
Time : 147.3173

Validating
Epoch 3
Loss : 0.5505
Acc : 72.1736
Time : 26.0032

Training
Epoch 4
Loss : 0.5721
Acc : 70.37
Time : 146.2402

Validating
Epoch 4
Loss : 0.5522
Acc : 72.0342
Time : 26.3556

Training
Epoch 5
Loss : 0.5706
Acc : 70.65
Time : 146.664

Validating
Epoch 5
Loss : 0.5514
Acc : 71.9944
Time : 25.9274


# 7) Hãy huấn luyện một mô hình small (chẳng hạn MobileNetV3) không sử dụng pretrained model trên 1 epochs.


In [ ]:
from torchvision.models import mobilenet_v3_small
# If do not use pretrain model do not set pretrained = True
model=mobilenet_v3_small()
model.classifier._modules['3'] = nn.Sequential(
    nn.Linear(1024, 1, bias = True),
    nn.Sigmoid()
)

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# Loading model to device
model.to(device)

# No of epochs 
epochs = 1

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, best_val_acc, "mobilenet_v2")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Training
Epoch 1
Loss : 0.6341
Acc : 63.555
Time : 153.0898

Validating
Epoch 1
Loss : 0.6618
Acc : 63.117
Time : 26.1131


# 8) Coi ResNet18 là Teacher và MobileNet là Student. Huấn luyện mô hình Student theo phương pháp Knowledge Distillation. 

Theo đó chúng ta cần sử dụng các gợi ý là giá trị dự báo $\mathbf{y}_t$ từ mô hình teacher để hướng dẫn giá trị dự báo $\mathbf{y}_s$ từ mô hình student thông qua hàm mất mát $\mathcal{L}_{distill}()$. Bên cạnh đó huấn luyện song song mô hình student với nhãn thực tế $\mathbf{y}$ thông qua $\mathcal{L}_{student}()$.

Kết hợp tổng của student loss và distill loss theo công thức:

$$\mathcal{L}_{final}(\mathbf{y}, \mathbf{y}_s, \mathbf{y}_t) = \alpha \mathcal{L}_{student}(\mathbf{y}, \mathbf{y}_s) + \beta \mathcal{L}_{distill}(T(\mathbf{y}_t), T(\mathbf{y}_s))$$
 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def loss_fn_kd(outputs, labels, teacher_outputs, alpha=0.1, temperature=3):
    """
    Compute the knowledge-distillation (KD) loss given outputs, labels.
    "Hyperparameters": temperature and alpha
    NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
    and student expects the input tensor to be log probabilities!
    """
    T = temperature
    student_loss = F.cross_entropy(outputs, labels)
    distill_loss = nn.KLDivLoss()(F.softmax(outputs/T, dim=1),
                             F.softmax(teacher_outputs/T, dim=1))
    KD_loss = student_loss * (1. - alpha) +  distill_loss * (alpha * T * T)

    return KD_loss

In [ ]:
def train_one_epoch_kd(train_data_loader, teacher, student):
    '''
      train_data_loader: data loader of knowledge distillation
      teacher: teacher model that is bigger one and needs to transfer knowledge to student. 
      student: student model that is smaller one and get knowledge from teacher.
    '''
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()

    student.train()
    teacher.eval()
    for images, labels in train_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Reseting Gradients
        optimizer.zero_grad()
        
        #Forward
        output_teacher_batch = teacher(images)
        output_batch = student(images)
        #Calculating Loss
        _loss = loss_fn_kd(output_batch, labels, output_teacher_batch)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy on student
        acc = accuracy(output_batch, labels)
        epoch_acc.append(acc)
        
        #Backward
        _loss.backward()
        optimizer.step()
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

# Learning Rate Scheduler: Learning rate decay double time after each 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# No of epochs 
epochs = 5

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch_kd(train_data_loader, teacher, model)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, best_val_acc, "mobilenet_v3_small")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."



Training
Epoch 1
Loss : 0.0
Acc : 70.705
Time : 170.9411

Validating
Epoch 1
Loss : 0.552
Acc : 72.0541
Time : 25.8896

Training
Epoch 2
Loss : 0.0
Acc : 70.33
Time : 170.6852

Validating
Epoch 2
Loss : 0.552
Acc : 72.1338
Time : 25.813

Training
Epoch 3
Loss : 0.0
Acc : 70.29
Time : 170.593

Validating
Epoch 3
Loss : 0.5489
Acc : 72.3328
Time : 25.6357

Training
Epoch 4
Loss : 0.0
Acc : 70.26
Time : 170.9058

Validating
Epoch 4
Loss : 0.5512
Acc : 72.1736
Time : 25.7802


KeyboardInterrupt: ignored

# 9) Áp dụng thêm các kĩ thuật data augmentation kết hợp ảnh khác nhãn để tạo thành nhãn mềm và huấn luyện cải thiện tiếp model student.


There are many resource we can use CutMix and MixUp:

* Pytorch: Package [timm](https://fastai.github.io/timmdocs/augmentation), [CutMix - clovaai](https://github.com/clovaai/CutMix-PyTorch)

* Keras: [cutmix](https://keras.io/examples/vision/cutmix/), [mixup](https://keras.io/examples/vision/mixup/)


In [ ]:
def rand_bbox(size = (None, 3, 224, 224), lam=0.75):
    '''
    size: input size of batch
    lam: ratio of image area that is remained.
    '''
    W = size[2]
    H = size[3]
    # cut with and height according to lambda
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # bounding box return
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [ ]:
def train_one_epoch_cutmix(train_data_loader, model):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()

    model.train()

    for images, labels in train_data_loader:
        r = np.random.rand(1)
        cutmix_prob = 0.33
        beta = 1 # beta distribution parameters of cutmix area
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        #Reseting Gradients
        optimizer.zero_grad()
        if beta > 0 and r < cutmix_prob:
            # generate mixed sample
            lam = np.random.beta(beta, beta)
            # randomly shuffling batch
            rand_index = torch.randperm(images.size()[0]).cuda()
            labels_a = labels
            labels_b = labels[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(images.size(), lam)
            # mixture crop image into original image.
            images[:, :, bbx1:bbx2, bby1:bby2] = images[rand_index, :, bbx1:bbx2, bby1:bby2]
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.size()[-1] * images.size()[-2]))
            #Forward
            preds = model(images)
            #Calculating Loss
            _loss = criterion(preds, labels_a) * lam + criterion(preds, labels_b) * (1. - lam)
        else:
            # compute output
            preds = model(images)
            _loss = criterion(preds, labels)
        
        loss = _loss.item()
        epoch_loss.append(loss)
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
        
        #Backward
        _loss.backward()
        optimizer.step()
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

# Learning Rate Scheduler: Learning rate decay double time after each 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# No of epochs 
epochs = 5

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch_cutmix(train_data_loader, model)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, best_val_acc, "mobilenet_v3_small")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#


Training
Epoch 1
Loss : 0.5901
Acc : 68.24
Time : 156.4167

Validating
Epoch 1
Loss : 0.5594
Acc : 71.258
Time : 26.2529

Training
Epoch 2
Loss : 0.5567
Acc : 71.565
Time : 151.8589

Validating
Epoch 2
Loss : 0.4728
Acc : 77.1099
Time : 26.21

Training
Epoch 3
Loss : 0.5357
Acc : 73.475
Time : 150.3429

Validating
Epoch 3
Loss : 0.491
Acc : 75.7763
Time : 25.8716

Training
Epoch 4
Loss : 0.5196
Acc : 74.25
Time : 150.0774

Validating
Epoch 4
Loss : 0.4343
Acc : 80.215
Time : 25.9817

Training
Epoch 5
Loss : 0.4994
Acc : 75.32
Time : 150.1104

Validating
Epoch 5
Loss : 0.4402
Acc : 78.0653
Time : 25.8289


# 10) Giả định cần huấn luyện tiếp mô hình student với các dữ liệu mới chưa được gán nhãn. Hãy xây dựng một kĩ thuật lựa chọn mẫu dựa trên đánh giá uncertainty.

Xem thêm [Active Learning](https://colab.research.google.com/drive/1m3K9-_u468O9hA_4S-uTbbafiXAPxt86?usp=sharing)